In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

tmpfiles = '/content/tmp/'
train_tmp_folder = tmpfiles+'train_embeddinds/'
test_tmp_folder = tmpfiles+'test_embeddinds/'
val_tmp_folder = tmpfiles+'val_embeddinds/'

import os.path
from os import path

if path.exists(tmpfiles) == False:

  os.mkdir(tmpfiles)
else:
  if path.exists(train_tmp_folder) == False or path.exists(test_tmp_folder) == False or path.exists(val_tmp_folder) == False:
    try:
      os.mkdir(train_tmp_folder )
      os.mkdir(test_tmp_folder )
      os.mkdir(val_tmp_folder )
    except Exception:
      pass

!ls

#%cd /content/gdrive/MyDrive/ThermostabilityProject/
!pip install transformers tqdm
%cd /content/gdrive/MyDrive/ThermostabilityProject/


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
gdrive	sample_data  tmp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/gdrive/MyDrive/ThermostabilityProject


In [2]:
import pandas as pd
import numpy as np
import torch, torch.nn
from transformers import BertModel, BertTokenizer, pipeline
import re, gc, os
import requests
from tqdm.auto import tqdm

In [3]:
def train_test_validation_splits(df):
    test,train_tot = df.loc[df['set']=='test'],df.loc[df['set']=='train']
    train, val = train_tot.loc[train_tot['validation']!=True], train_tot.loc[train_tot['validation']==True]
    return train,val, test

def build_batch_iterator_sequences(sequences_total, batch_size):

    long_list = sequences_total
    sub_list_length = batch_size
    sub_lists = [
        long_list[i : i + sub_list_length]
        for i in range(0, len(long_list), sub_list_length)
    ]
    return sub_lists

def get_features_from_embeddings(embeddings, attention_mask):
    features = [] 
    for seq_num in range(len(embeddings)):
          seq_len = (attention_mask[seq_num] == 1).sum()
          seq_emd = embeddings[seq_num][1:seq_len-1]
          features.append(seq_emd)

def embed_dataset(dataset_seqs, model, tokenizer, device, path, shift_left = 1, shift_right = -1):
  inputs_embedding = []

  cont=0
  
  for sample in tqdm(dataset_seqs):
    with torch.no_grad():
      ''' As we are using it in unsupervised manner, we are ignoring the CLS token and so, what's why 
          we are ignoring the special tokens (inspect better in huggingface why the three values in the output)'''
      #ids = tokenizer.batch_encode_plus(sample, add_special_tokens=True, padding=True, is_split_into_words=True, return_tensors="pt")      
      #ids = tokenizer.batch_encode_plus(sample, add_special_tokens=False, padding=True, is_split_into_words=True, return_tensors="pt")
      #embedding = model(input_ids=ids['input_ids'].to(device))[0]
      #inputs_embedding.append(embedding.detach().cpu().numpy())
      
      #ids = tokenizer.encode_plus(list(sample.strip()), add_special_tokens=True, padding=True, 
      #                                                         return_attention_mask = True, return_tensors="pt")
      #embedding = model(input_ids=ids['input_ids'].reshape(-1,1).to(device), attention_mask = ids['attention_mask'].reshape(-1,1).to(device))[0]
      #np.savez(path + str(cont)+'.npz',  embedding.detach().cpu().numpy() )
      #cont+=1
  


      ''' Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)'''
      #import pdb; pdb.set_trace()
      sequences_Example = sample #["A E T C Z A O","S K T Z P"]

      sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

      '''. Tokenize, encode sequences and load it into the GPU if possibile'''

      ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True)

      input_ids = torch.tensor(ids['input_ids']).to(device)
      attention_mask = torch.tensor(ids['attention_mask']).to(device)

      ''' Extracting sequences' features and load it into the CPU if needed'''

      with torch.no_grad():
          embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

      #import pdb; pdb.set_trace()
      embedding = embedding.detach().cpu().numpy()

      #embedding = get_features_from_embeddings(embedding)


      np.savez(path + str(cont)+'.npz',  embedding )
      cont+=1

      #del ids, embedding
      #gc.collect()
      #torch.cuda.empty_cache()


  return inputs_embedding




In [4]:
path_meltome = 'datasets/FLIP/splits/meltome/splits/mixed_split.csv'

splits_meltome = pd.read_csv(path_meltome, sep=',')
split_meltome = splits_meltome[splits_meltome['sequence'].str.len()>= 50]
splits_meltome['sequence']=splits_meltome['sequence'].str.join(" ")
train, val, test = train_test_validation_splits(splits_meltome)
print('done')

done


In [5]:
len(splits_meltome['sequence'].to_list())

27951

In [6]:
!pip install accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#from accelerate import init_empty_weights
#from accelerate import infer_auto_device_map
max_memory_mapping = {0: "7GB", 'cpu': "8GB"}

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertModel.from_pretrained("Rostlab/prot_bert", low_cpu_mem_usage=True, max_memory=max_memory_mapping)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = model.to(device=device)
model = model.eval()
if torch.cuda.is_available():
  model = model.half()

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def pooling_and_final_representation(path):
  # Due to npz compression on each embedded sequence
  #import pdb; pdb.set_trace()
  sample = np.load(path)
  embedded_seq = torch.from_numpy( sample[sample.files[0]] ).to(torch.float32)
  #embedded_seq = embedded_seq.view(embedded_seq.size(0), -1).permute(1,0)
  embedded_seq = embedded_seq.permute(0,2,1)
  GlobalAvgpooling = torch.nn.AvgPool1d(kernel_size = embedded_seq.size(2) , stride = 1, padding = 0)
  pooled_seq = GlobalAvgpooling(embedded_seq)

  return pooled_seq #pooled_seq.permute(1,0)

#the test

def pooled_set_of_sequences(folder_path):
  #import pdb; pdb.set_trace()
  list_embeddings = []
  iter_paths = os.listdir(folder_path)
  for path in tqdm(iter_paths):
    list_embeddings.append( pooling_and_final_representation(folder_path + path) )

  #import pdb;pdb.set_trace()
  representation = torch.stack(list_embeddings)
  representation = representation.view(representation.size(0),-1)
  return representation

#train_embeddings = pooled_set_of_sequences(train_tmp_folder)
#test_embeddings = pooled_set_of_sequences(test_tmp_folder)
#val_embeddings = pooled_set_of_sequences(val_tmp_folder)

In [9]:
dir_pre_generated_embeddings = ''
path_train= dir_pre_generated_embeddings+'train_set_embeddings.pth'
path_test= dir_pre_generated_embeddings+'test_set_embeddings.pth'
path_val= dir_pre_generated_embeddings+'val_set_embeddings.pth'


if os.path.isfile(path_train)==False and os.path.isfile(path_test)==False and os.path.isfile(path_val)==False:
    #if path.exists(train_tmp_folder) == False or path.exists(test_tmp_folder) == False or path.exists(val_tmp_folder) == False:
    batch_size = 1
    batch_train = build_batch_iterator_sequences(train.sequence.to_list(), batch_size)
    batch_test = build_batch_iterator_sequences(test.sequence.to_list(), batch_size)
    batch_val = build_batch_iterator_sequences(val.sequence.to_list(), batch_size)
    
    
    print ("Creating Embeddings...")
    embed_dataset(batch_train, model, tokenizer, device, train_tmp_folder)
    embed_dataset(batch_test, model, tokenizer, device, test_tmp_folder)
    embed_dataset(batch_val, model, tokenizer, device, val_tmp_folder)
    print ("DONE!!...")

    print ("Loading Premade Embeddings...")

    train_embeddings = pooled_set_of_sequences(train_tmp_folder)
    torch.save(train_embeddings, 'train_set_embeddings.pth')
    test_embeddings = pooled_set_of_sequences(test_tmp_folder)
    torch.save(test_embeddings, 'test_set_embeddings.pth')
    val_embeddings = pooled_set_of_sequences(val_tmp_folder)
    torch.save(val_embeddings, 'val_set_embeddings.pth')
    print ("Loaded")
else:
    print ("Loading Premade Embeddings...")
    train_embeddings = torch.load('train_set_embeddings.pth')
    test_embeddings = torch.load('test_set_embeddings.pth')
    val_embeddings = torch.load('val_set_embeddings.pth')
    print ("Loaded")




Creating Embeddings...


  0%|          | 0/22335 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 7>:16                                                                             │
│ in embed_dataset:61                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py:1020 in        │
│ forward                                                                                          │
│                                                                                                  │
│   1017 │   │   │   inputs_embeds=inputs_embeds,                                                  │
│   1018 │   │   │   past_key_values_length=past_key_values_length,                                │
│   1019 │   │   )                                                                                 │
│ ❱ 1020 │   │   encoder_outputs = self.encoder(                                                   │
│   1021 │   │   │   embedding_output,                                                             │
│   1022 │   │   │   attention_mask=extended_attention_mask,                                       │
│   1023 │   │   │   head_mask=head_mask,                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py:610 in forward │
│                                                                                                  │
│    607 │   │   │   │   │   encoder_attention_mask,                                               │
│    608 │   │   │   │   )                                                                         │
│    609 │   │   │   else:                                   

In [ ]:
!ls -1 /content/tmp

In [ ]:
import torch.utils.data as data_utils

batch_size=50


trainDataset = data_utils.TensorDataset(train_embeddings, torch.from_numpy( train.target.to_numpy() ))
testDataset = data_utils.TensorDataset(test_embeddings, torch.from_numpy( test.target.to_numpy() ))
valDataset = data_utils.TensorDataset(val_embeddings, torch.from_numpy( val.target.to_numpy() ))

train_loader = data_utils.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(testDataset, batch_size=batch_size, shuffle=True)
val_loader = data_utils.DataLoader(valDataset, batch_size=batch_size, shuffle=True)

In [ ]:
class Flatten(torch.nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    
    def forward(self, input):
        return input.view(input.size(0), -1)   

class regressionHead(torch.nn.Module):

    def __init__(self, kernel):
        super(regressionHead, self).__init__()
        #self.input_shape = input_shape # pos0 = #channels, pos1 = #diagonal comps, or viseversa


        self.FFNN = torch.nn.Sequential(
            Flatten(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(), #nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(), #nn.ReLU(),
            torch.nn.Linear(512, 1),
            torch.nn.ReLU()
        )
        
        #self.encoder.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.kaiming_normal_(module.weight)

    def forward(self, x):
        #x = self.pooling(x).mean(dim=0)
        z = self.FFNN(x)
        return z #z.permute(0,2,1) 

In [ ]:
def train_LLMRegresor(train_iterator, val_iterator, model, device, criterion, optimizer, epoch_num):
    
        
        val_loss = []
        
        model.to(device)

        for epoch in range(epoch_num):
            model.train() 
            for i, (input, label) in enumerate(train_iterator):
                optimizer.zero_grad()
                
                input = input.to(dtype=torch.float32, device=device)
                label = label.to(dtype=torch.float32, device=device) 
                out = model(input)
                loss = criterion(out, label.unsqueeze(-1))
                loss.backward() 
                optimizer.step()

            with torch.no_grad(): # evaluate validation loss here 

                model.eval()
                val_loss_epochs = []

                for (inputval, labelval) in val_iterator:
                    
                    inputval = inputval.to(device)
                    labelval = labelval.to(device)
                    outval = model(inputval)
                    lossval = criterion(outval, labelval.unsqueeze(-1)) # Calculate validation loss 
                    val_loss_epochs.append(lossval.item())

                val_loss_epoch = np.mean(val_loss_epochs)
                val_loss.append(round(val_loss_epoch, 3))

            print('epoch: %d loss: %.3f val loss: %.3f' % (epoch + 1, loss.item(), val_loss_epoch))

            '''# evalutate whether validation loss is dropping; if not, stop
            if epoch > 21:
                if val_loss[-1] >= np.min(val_loss[:-20]): 
                    print('Finished training at epoch {0}'.format(epoch))
                    return epoch'''

In [ ]:
model = regressionHead(kernel=3)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss= torch.nn.SmoothL1Loss()
epochs = 500
train_LLMRegresor(train_loader, val_loader, model, device, loss, opt, epochs)

In [ ]:
test_set, test_labels = test_loader.dataset.tensors

In [ ]:
import math
test_set = test_set.to(dtype=torch.float32, device=device)
test_labels = test_labels.to(dtype=torch.float32, device=device) 
outcome = model(test_set)

print(outcome, test_labels)
#import pdb; pdb.set_trace()

x = np.array( list(range(0, len(outcome))) )
y= torch.abs(outcome.squeeze(1)-test_labels).detach().cpu().numpy()

import matplotlib.pyplot as plt

plt.bar( x, y )


In [ ]:
print(outcome.squeeze(1)[1000:1300])
print(test_labels[1000:1300])

In [ ]:
loss_test = loss(outcome, test_labels.unsqueeze(-1))
print('MSE: ' + str(loss_test))
print('RMSE: ' + str(torch.sqrt(loss_test)))

In [ ]:



'''sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)
print(output)'''